In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings,ChatHuggingFace ,HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

In [ ]:
load_dotenv()

Fetching Data from YT API


In [ ]:
def fetched_transcript_Formater(inputs):
    transcripter = YouTubeTranscriptApi()
    

    # Data Extraction ->Data loading
    transcript_output =  transcripter.fetch(
    video_id= inputs["video_id"],
    languages=['hi', 'en']
    ).to_raw_data()
    output_string = " "
    for i in transcript_output :
        output_string += (" " + i.get('text'))
    print("Video Transcripted")
    return output_string.replace("\n" , " ")
    

# Creatring the instance of Trancript Class




Splitting and chunking 

In [ ]:
def split_text(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap  = 200,

    )
    chunks = splitter.create_documents([text])
    print("length of chunks" , len(chunks))
    return chunks
# print("chunks:: ", chunks)

Vector Store Creation and Populating Step

In [ ]:
def vector_Store_create(docs):
    embedding_model = HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-mpnet-base-v2"
)
    vectorDb  = Chroma.from_documents(
    collection_name= "myYtDb",
    embedding = embedding_model,
    documents= docs
)
   
    print("vector_Store_create", vectorDb)
   
    retriever = vectorDb.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k' : 4}
)
    
    return {'retriever': retriever, 'vs': vectorDb}
   
    
    
 


Augmentation Stage

In [ ]:
tempalte = PromptTemplate(
    template=  """
    You are a helpful AI Assistant.
    Answere only from the transcript context.
    If context is insufficent just prompt i am unable to figure out amswere.
    {context}
    question : {question}
    """ ,
    input_variables= ['context' , 'question']
)


print("Tempalte Augmented" , tempalte)

In [ ]:

retriever = vectorDb.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k' : 4}
)
    


LLM Defination


In [ ]:
llm  = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1-0528",
    task="text-generation"
)
model = ChatHuggingFace(
    llm = llm 
)

Build PipeLine

In [ ]:
video_id  = "1bUy-1hGZpI" 
user_input_question = input("Enter the question please : ")
pipeLine = (
    RunnableLambda(fetched_transcript_Formater)|
    RunnableLambda(split_text)|
    RunnableLambda(vector_Store_create)|    
    RunnableLambda(retriever.invoke(user_input_question))
    # RunnableLambda(lambda docs: "".join(doc.page_content for doc in docs))|
    # tempalte|
    # model



)
pipeLine.invoke({"video_id" : video_id , "user_input_question" : user_input_question , "question" : user_input_question})

In [ ]:
video_id  = "1bUy-1hGZpI" 
user_input_question = input("Enter the question please : ")
pipeLine = (
    RunnableLambda(fetched_transcript_Formater)
    )
pipeLine.invoke({"video_id" : video_id})

ImportError: cannot import name 'StuffDocumentsChain' from 'langchain.chains.combine_documents' (c:\Users\bansa\Desktop\GenAi\myenv\Lib\site-packages\langchain\chains\combine_documents\__init__.py)